In [ ]:
import polars as pl
import pandas as pd
import numpy as np
import catboost
import os
from datetime import date, timedelta
from itertools import combinations

from local_utils import *
import lightgbm as lg

test_start_date = date(2024, 8, 1)
val_start_date = date(2024, 7, 1)
val_end_date = date(2024, 7, 31)
train_end_date = date(2024, 6, 30)
data_path = "C:\\Users\\Admin\\Desktop\\AIM 2сем\\ML2\\hw2"

search_history = pl.scan_parquet(os.path.join(data_path, 'cluster_search/*.parquet')).collect()

In [6]:
search_history = search_history.to_pandas()

In [47]:
import pandas as pd
from collections import Counter

sh_topd_filtered = search_history[search_history['cluster'] == -1].copy()

all_words = []
for query in sh_topd_filtered['search_query']:
    words = query.split()
    all_words.extend(words)

unique_words = set(all_words)

word_counts = Counter(all_words)

# Получаем список кортежей (слово, количество) и сортируем по убыванию частоты
sorted_word_counts = word_counts.most_common()

# Извлекаем только слова из отсортированного списка
top_words = [word for word, count in sorted_word_counts]

print(f"Всего уникальных слов: {len(unique_words)}")
print("Топ самых частых слов:")
print(top_words[:150])  # Выводим топ-25 слов

Всего уникальных слов: 599032
Топ самых частых слов:
['на', 'с', 'набор', 'от', 'в', 'мои', 'озон', 'заказы', '1', 'лента', 'веган', 'и', '4', '2', '3', 'ozon', 'товар', 'уцененный', '5', 'по', 'из', '8', 'а4', 'товары', 'крахмал', 'мини', 'за', 'аккумулятор', 'кг', 'без', 'покупки', '10', 'марта', 'gold', 'nestogen', 'персил', 'пава', 'domestos', 'козинаки', 'kix', 'мешки', 'к', 'красный', 'one', 'черный', 'ручки', 'патчи', '6', '7', 'л', 'см', 'купить', 'яица', 'состски', 'под', 'сердечки', 'про', 'марс', 'терка', 'рафаэлло', 'слива', 'бенто', 'альметте', 'ватрушка', 'круасан', 'дюшес', 'мм', 'bang', 'скитлс', 'мл', 'трюфель', '100', 'фумигатор', 'монитор', 'прошутто', 'ушастый', 'веник', 'халуми', 'милки', 'удон', 'нянь', 'электронная', 'аппарат', 'черкизово', 'ferrero', 'чери', 'новотерская', 'maggi', 'горки', 'хачапури', 'мой', 'маракуйя', 'крупы', 'aravia', 'аптека', 'лёд', 'киткат', 'оливье', 'and', '30', 'белый', 'капучинатор', 'маргарин', 'purina', 'сигарета', 'романо', 'замок

In [50]:
print(top_words[:500])

['на', 'с', 'набор', 'от', 'в', 'мои', 'озон', 'заказы', '1', 'лента', 'веган', 'и', '4', '2', '3', 'ozon', 'товар', 'уцененный', '5', 'по', 'из', '8', 'а4', 'товары', 'крахмал', 'мини', 'за', 'аккумулятор', 'кг', 'без', 'покупки', '10', 'марта', 'gold', 'nestogen', 'персил', 'пава', 'domestos', 'козинаки', 'kix', 'мешки', 'к', 'красный', 'one', 'черный', 'ручки', 'патчи', '6', '7', 'л', 'см', 'купить', 'яица', 'состски', 'под', 'сердечки', 'про', 'марс', 'терка', 'рафаэлло', 'слива', 'бенто', 'альметте', 'ватрушка', 'круасан', 'дюшес', 'мм', 'bang', 'скитлс', 'мл', 'трюфель', '100', 'фумигатор', 'монитор', 'прошутто', 'ушастый', 'веник', 'халуми', 'милки', 'удон', 'нянь', 'электронная', 'аппарат', 'черкизово', 'ferrero', 'чери', 'новотерская', 'maggi', 'горки', 'хачапури', 'мой', 'маракуйя', 'крупы', 'aravia', 'аптека', 'лёд', 'киткат', 'оливье', 'and', '30', 'белый', 'капучинатор', 'маргарин', 'purina', 'сигарета', 'романо', 'замок', 'люстра', 'обед', 'машинки', 'обогреватель', 'прин

In [ ]:
search_history.loc[search_history['search_query'].str.contains('ваз', case=False, na=False), 'cluster'] = 20
search_history.loc[search_history['search_query'].str.contains('nivea', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('monterra', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('гуляш', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('равиоли', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('хурма', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('облепиха', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('пергамент', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('палтус', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('инструменты', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('фейри', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('фона', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('вешалки', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('смекта', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('горшок', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('швепс', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('поп', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('dolce', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('роял', case=False, na=False), 'cluster'] = 8
search_history.loc[search_history['search_query'].str.contains('пикник', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('розетка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('ремит', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('авакадо', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('lindt', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('гантели', case=False, na=False), 'cluster'] = 21


search_history.loc[search_history['search_query'].str.contains('head', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('салями', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('эспумизан', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('finish', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('велком', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('панировка', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('балык', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('перепелка', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('ice', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('бублики', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('бокал', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('velle', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('matrix', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('нитки', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('similac', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('пазл', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('yelli', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('infinix', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('тостер', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('бепантен', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('кабинет', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('поке', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('помадка', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('консилер', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('азелит', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('шприц', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('анальная', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('кит', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('лоток', case=False, na=False), 'cluster'] = 8
search_history.loc[search_history['search_query'].str.contains('мойва', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('спирт', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('вешалка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('hdmi', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('махеев', case=False, na=False), 'cluster'] = 14
search_history.loc[search_history['search_query'].str.contains('крючки', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('ацидофилин', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('псыж', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('край', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('эскимо', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('роше', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('holika', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('член', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('казан', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('конфитюр', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('туалет', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('простынь', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('паэлья', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('kitkat', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('электронная', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('аппарат', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('черкизово', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('ferrero', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('чери', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('новотерская', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('maggi', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('горки', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('хачапури', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('мой', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('маракуйя', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('крупы', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('aravia', case=False, na=False), 'cluster'] = 19 
search_history.loc[search_history['search_query'].str.contains('аптека', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('лёд', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('киткат', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('оливье', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('капучинатор', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('маргарин', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('purina', case=False, na=False), 'cluster'] = 8
search_history.loc[search_history['search_query'].str.contains('сигарета', case=False, na=False), 'cluster'] = 20
search_history.loc[search_history['search_query'].str.contains('романо', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('замок', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('люстра', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('обед', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('машинки', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('обогреватель', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('принглс', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('ромашка', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('марципан', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('инжир', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('триммер', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('фигурка', case=False, na=False), 'cluster'] = 20
search_history.loc[search_history['search_query'].str.contains('мелатонин', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('дрип', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('санокс', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('solvie', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('шар', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('бульмени', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('конверт', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('fairy', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('веган', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('крахмал', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('аккумулятор', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('nestogen', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('персил', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('пава', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('domestos', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('козинаки', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('kix', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('мешки', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('one', case=False, na=False), 'cluster'] = 8
search_history.loc[search_history['search_query'].str.contains('ручки', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('патчи', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('яица', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('состски', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('сердечки', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('марс', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('терка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('рафаэлло', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('слива', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('бенто', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('альметте', case=False, na=False), 'cluster'] = 7
search_history.loc[search_history['search_query'].str.contains('ватрушка', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('круасан', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('дюшес', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('bang', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('скитлс', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('трюфель', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('фумигатор', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('монитор', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('прошутто', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('ушастый', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('веник', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('халуми', case=False, na=False), 'cluster'] = 7
search_history.loc[search_history['search_query'].str.contains('милки', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('удон', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('нянь', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('18+', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('шейкер', case=False, na=False), 'cluster'] = 21
search_history.loc[search_history['search_query'].str.contains('диск', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('крылышки', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('урбеч', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('чашка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('merci', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('ржаной', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('фери', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('манты', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('килька', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('массажер', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('лопатка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('ежедневник', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('лекарства', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('насадка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('estel', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('доктор', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('стрепсилс', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('мерси', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('коробки', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('лисички', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('twix', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('rocher', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('розжиг', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('миска', case=False, na=False), 'cluster'] = 8
search_history.loc[search_history['search_query'].str.contains('viola', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('пепси', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('pusy', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('ирригатор', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('сальчичон', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('пазлы', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('шуруповерт', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('галеты', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('lipton', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('подушечки', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('lovular', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('силит', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('cat', case=False, na=False), 'cluster'] = 8
search_history.loc[search_history['search_query'].str.contains('vichy', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('шрирача', case=False, na=False), 'cluster'] = 14
search_history.loc[search_history['search_query'].str.contains('пробка', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('mixit', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('газовая', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('коржи', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('панчо', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('nike', case=False, na=False), 'cluster'] = 12
search_history.loc[search_history['search_query'].str.contains('домик', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('таблетница', case=False, na=False), 'cluster'] = 13

In [ ]:
'''cluster_mapping = {
    1: ['borjomi', 'пармалат', 'теос', 'nemoloko', 'имунеле', 'таорог', 'эконива', 'гхи', 'масле', 'савушкин'],
    2: ['закуска', 'жевачка', 'чтпсы', 'rex', 'любятово', 'щербет', 'mirel', 'шоколад', 'баунти', 'наполеон', 
        'читос', 'fitnesshock', 'shock', 'кето', 'аленка', 'чоко', 'фри', 'бабл', 'тирамису'],
    3: ['макаруны', 'кимчи', 'клетчатка', 'псиллиум', 'nordic', 'мистраль', 'увелка', 'биг', 'птитим', 'кус', 
        'корнерсы', 'чиа', 'отруби', 'кускус', 'плов', 'ясно', 'травы'],
    4: ['абрикосы', 'архыз', 'халапеньо', 'смородина', 'черника', 'банка', 'помело', 'грейпфрут', 'мак', 'фундук',
       'бобы', 'каперсы', 'орешки', 'чука', 'лечо', 'сорбет', 'маринованные'],
    5: ['energy', 'мажитэль', 'nescafe', 'tea', 'greenfield', 'lavazza', 'evervess', 'adrenaline', 'шампанское',
       'бокалы', 'актибио', 'пиву', 'блю', 'pepper', 'rich', 'bushido', 'burn', 'нарзан'],
    6: ['савоярди', 'эклер', 'пончики', 'гренки', 'слойка', 'брауни', 'лепешки', 'пирожки', 'безе', 'запеканка',
       'оладьи'],
    7: ['natura', 'маасдам', 'сиртаки', 'чечил', 'бри'],
    8: ['pi', 'дог'],
    9: ['тефтели', 'перекус', 'сибас', 'телятина', 'штрудель', 'окорок', 'докторская', 'язык', 'жареный', 'хинкали',
       'окраина', 'вязанка', 'шницель', 'дамплинги', 'крылья', 'сервелат', 'фуэт', 'заморозка', 'чевапчичи', 'hi',
       'буженина'],
    10: ['папка', 'liebe', 'фломастеры', 'ваниш', 'мешок', 'синергетик', 'таз', 'жалюзи', 'сады', 'обои', 'фигурки',
        'маркер', 'лак', 'тазик', 'листы', 'шланг', 'брелок', 'кассеты', 'органайзер', 'полка', 'вернель', 'мебель',
        'кухонные', 'разделочная', 'мух', 'кухонный', 'бутылка', 'приготовления', 'пост', 'гладильная', 'тарелки',
        'корзина', 'бокс', 'расческа', 'термос', 'пемолюкс', 'спички', 'зажигалка', 'насос', 'ковер', 'очиститель',
        'ножи', 'блюда', 'труб', 'надувной', 'grass', 'пила', 'комод', 'садовая', 'история', 'засоров', 'тапки',
        'кольцо', 'тараканов'],
    11: ['lol', 'мама', 'kabrita', 'кубики', 'momi', 'baby', 'semper', 'кабрита', 'малютка', 'nutrilak', 'pampers',
        'беби', 'малоежка', 'gerber', 'коляска', 'merries', 'хот', 'комфорт'],
    12: ['дождевик'],
    13: ['ти', 'кашля', 'just', 'эвалар', 'ополаскиватель', 'термометр', 'снуп', 'полоски', 'хлоргексидин', 'солнышко',
        'пантенол', 'мирамистин', 'раствор', 'acuvue', 'bio', 'форте', 'терафлю', 'полисорб', 'прыщей', 'аквафор',
        'тонометр', 'градусник'],
    14: ['heinz', 'тар'],
    15: ['аккумуляторная', 'отпариватель', 'mio', 'алиса', 'станция', 'вафельница', 'роутер', 'яндекс', 'диффузор',
        'робот', 'автомобильный'],
    17: ['земля', 'дача'],
    19: ['green', 'dove', 'лепестки', 'чистая', 'лубрикант', 'пудра', 'серьги', 'карандаш', 'накладные', 'косметичка',
        'браслет', 'доя', 'ногти', 'линия', 'bioderma'],
    20: ['табачные', 'авто', 'gillette', 'игры', 'автомобиль', 'дорожный'],
    21: ['korner', 'стевия', 'креатин', 'спорт', 'fit', 'мяч', 'fitnesshock']
}'''

In [ ]:
'''cluster_mapping = {
    1: ['пармалат', 'теос', 'nemoloko', 'имунеле', 'таорог', 'эконива', 'гхи', 'масле', 'савушкин',
        'monterra', 'ацидофилин', 'эскимо', 'маргарин', 'viola', 'яица', 'урбеч'],
    2: ['закуска', 'жевачка', 'чтпсы', 'rex', 'любятово', 'щербет', 'mirel', 'шоколад', 'баунти', 'наполеон',
        'читос', 'fitnesshock', 'shock', 'кето', 'аленка', 'чоко', 'фри', 'бабл', 'равиоли', 'роше', 'конфитюр',
        'kitkat', 'ferrero', 'марс', 'рафаэлло', 'бенто', 'скитлс', 'milky', 'merci', 'мерси', 'twix', 'подушечки',
        'панчо', 'пикник', 'solvie', 'веган', 'крупы', 'lindt', 'роял'],
    3: ['макаруны', 'кимчи', 'клетчатка', 'псиллиум', 'nordic', 'мистраль', 'увелка', 'биг', 'птитим', 'кус',
        'корнерсы', 'чиа', 'отруби', 'кускус', 'плов', 'ясно', 'травы', 'паэлья', 'yelli', 'удон'],
    4: ['абрикосы', 'архыз', 'халапеньо', 'смородина', 'черника', 'банка', 'помело', 'грейпфрут', 'мак', 'фундук',
       'бобы', 'каперсы', 'орешки', 'чука', 'лечо', 'сорбет', 'маринованные', 'хурма', 'облепиха', 'чери', 'слива',
       'трюфель', 'ржаной', 'оливье', 'романо', 'инжир', 'крахмал'],
    5: ['borjomi', 'energy', 'мажитэль', 'nescafe', 'tea', 'greenfield', 'lavazza', 'evervess', 'adrenaline', 'шампанское',
       'бокалы', 'актибио', 'пиву', 'блю', 'pepper', 'rich', 'bushido', 'burn', 'нарзан', 'швепс', 'дюшес', 'пепси',
       'lipton', 'новотерская', 'лёд', 'drip'],
    6: ['савоярди', 'эклер', 'пончики', 'гренки', 'слойка', 'брауни', 'лепешки', 'пирожки', 'безе', 'запеканка',
       'оладьи', 'пергамент', 'ватрушка', 'круасан', 'галеты', 'коржи', 'хачапури', 'край'],
    7: ['natura', 'маасдам', 'сиртаки', 'чечил', 'бри', 'альметте', 'халуми'],
    8: ['pi', 'дог', 'purina', 'one', 'лоток', 'миска', 'cat', 'роял'],
    9: ['тефтели', 'перекус', 'сибас', 'телятина', 'штрудель', 'окорок', 'докторская', 'язык', 'жареный', 'хинкали',
       'окраина', 'вязанка', 'шницель', 'дамплинги', 'крылья', 'сервелат', 'фуэт', 'заморозка', 'чевапчичи', 'hi',
       'буженина', 'гуляш', 'палтус', 'велком', 'панировка', 'балык', 'перепелка', 'черкизово', 'maggi', 'обед',
       'бульмени', 'манты', 'килька', 'крылышки', 'прошутто', 'сальчичон', 'состски'],
    10: ['папка', 'liebe', 'фломастеры', 'ваниш', 'мешок', 'синергетик', 'таз', 'жалюзи', 'сады', 'обои', 'фигурки',
        'маркер', 'лак', 'тазик', 'листы', 'шланг', 'брелок', 'кассеты', 'органайзер', 'полка', 'вернель', 'мебель',
        'кухонные', 'разделочная', 'мух', 'кухонный', 'бутылка', 'приготовления', 'пост', 'гладильная', 'тарелки',
        'корзина', 'бокс', 'расческа', 'термос', 'пемолюкс', 'спички', 'зажигалка', 'насос', 'ковер', 'очиститель',
        'ножи', 'блюда', 'труб', 'надувной', 'grass', 'пила', 'комод', 'садовая', 'история', 'засоров', 'тапки',
        'кольцо', 'тараканов', 'инструменты', 'фейри', 'вешалки', 'горшок', 'розетка', 'вешалка', 'крючки', 'азелит',
        'санокс', 'конверт', 'fairy', 'ручки', 'терка', 'сердечки', 'фумигатор', 'ушастый', 'веник', 'диск', 'чашка',
        'массажер', 'лопатка', 'ежедневник', 'насадка', 'коробки', 'розжиг', 'газовая', 'силит', 'персил', 'kix',
        'мешки', 'domestos', 'казан', 'туалет', 'простынь', 'bang', 'монитор'],
    11: ['lol', 'мама', 'kabrita', 'кубики', 'momi', 'baby', 'semper', 'кабрита', 'малютка', 'nutrilak', 'pampers',
        'беби', 'малоежка', 'gerber', 'коляска', 'merries', 'хот', 'комфорт', 'поп', 'similac', 'пазл', 'нянь',
        'lovular', 'горки', 'машинки', 'шар', 'домик', 'пазлы'],
    12: ['дождевик', 'nike'],
    13: ['ти', 'кашля', 'just', 'эвалар', 'ополаскиватель', 'термометр', 'снуп', 'полоски', 'хлоргексидин', 'солнышко',
        'пантенол', 'мирамистин', 'раствор', 'acuvue', 'bio', 'форте', 'терафлю', 'полисорб', 'прыщей', 'аквафор',
        'тонометр', 'градусник', 'смекта', 'эспумизан', 'ромашка', 'мелатонин', 'лекарства', 'доктор', 'стрепсилс',
        'лисички', 'таблетница', 'аптека'],
    14: ['heinz', 'тар', 'махеев', 'шрирача'],
    15: ['аккумуляторная', 'отпариватель', 'mio', 'алиса', 'станция', 'вафельница', 'роутер', 'яндекс', 'диффузор',
        'робот', 'автомобильный', 'фона', 'infinix', 'тостер', 'hdmi', 'аккумулятор', 'ирригатор', 'шуруповерт',
        'триммер', 'капучинатор', 'электронная', 'аппарат', 'монитор'],
    17: ['земля', 'дача'],
    19: ['green', 'dove', 'лепестки', 'чистая', 'лубрикант', 'пудра', 'серьги', 'карандаш', 'накладные', 'косметичка',
        'браслет', 'доя', 'ногти', 'линия', 'bioderma', 'nivea', 'dolce', 'head', 'ice', 'matrix', 'консилер',
        'анальная', 'спирт', 'holika', 'член', '18+', 'estel', 'rocher', 'пробка', 'mixit', 'vichy', 'pusy',
        'aravia'],
    20: ['табачные', 'авто', 'gillette', 'игры', 'автомобиль', 'дорожный', 'vaz', 'сигарета', 'фигурка'],
    21: ['korner', 'стевия', 'креатин', 'спорт', 'fit', 'мяч', 'fitnesshock', 'гантели', 'шейкер']
}'''

In [53]:
import re

for cluster_id, keywords in cluster_mapping.items():
    pattern = '|'.join(map(re.escape, keywords))
    search_history.loc[search_history['search_query'].str.contains(pattern, case=False, na=False), 'cluster'] = cluster_id

In [ ]:
search_history.loc[search_history['search_query'].str.contains('borjomi', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('lol', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('папка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('liebe', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('ти', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('авто', case=False, na=False), 'cluster'] = 20
search_history.loc[search_history['search_query'].str.contains('закуска', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('pi', case=False, na=False), 'cluster'] = 8
search_history.loc[search_history['search_query'].str.contains('фломастеры', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('жевачка', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('мама', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('ваниш', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('чтпсы', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('мешок', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('dove', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('синергетик', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('таз', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('макаруны', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('kabrita', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('rex', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('пармалат', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('аккумуляторная', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('bioderma', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('жалюзи', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('сады', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('обои', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('щербет', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('дорожный', case=False, na=False), 'cluster'] = 20
search_history.loc[search_history['search_query'].str.contains('кашля', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('mirel', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('фигурки', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('лепестки', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('кимчи', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('тар', case=False, na=False), 'cluster'] = 14
search_history.loc[search_history['search_query'].str.contains('сердечки', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('безе', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('клетчатка', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('маркер', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('лак', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('смородина', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('тазик', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('листы', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('аленка', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('шланг', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('птиц', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('брелок', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('korner', case=False, na=False), 'cluster'] = 21
search_history.loc[search_history['search_query'].str.contains('кассеты', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('коляска', case=False, na=False), 'cluster'] = 11


In [ ]:
search_history.loc[search_history['search_query'].str.contains('energy', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('табачные', case=False, na=False), 'cluster'] = 20
search_history.loc[search_history['search_query'].str.contains('green', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('чоко', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('тефтели', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('heinz', case=False, na=False), 'cluster'] = 14
search_history.loc[search_history['search_query'].str.contains('kix', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('мешки', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('мажитэль', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('nescafe', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('tea', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('domestos', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('greenfield', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('козинаки', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('абрикосы', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('архыз', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('садовая', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('кубики', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('перекус', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('любятово', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('комод', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('автомобиль', case=False, na=False), 'cluster'] = 20
search_history.loc[search_history['search_query'].str.contains('карандаш', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('серьги', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('ванную', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('стевия', case=False, na=False), 'cluster'] = 21
search_history.loc[search_history['search_query'].str.contains('momi', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('блюда', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('just', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('линия', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('труб', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('органайзер', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('полка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('эвалар', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('ополаскиватель', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('сибас', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('baby', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('термометр', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('natura', case=False, na=False), 'cluster'] = 7
search_history.loc[search_history['search_query'].str.contains('снуп', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('вернель', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('чистая', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('пари', case=False, na=False), 'cluster'] = 2

In [ ]:
search_history.loc[search_history['search_query'].str.contains('савушкин', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('креатин', case=False, na=False), 'cluster'] = 21
search_history.loc[search_history['search_query'].str.contains('надувной', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('савоярди', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('отпариватель', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('grass', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('лубрикант', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('пила', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('халапеньо', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('хот', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('псиллиум', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('фри', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('мебель', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('телятина', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('nestogen', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('персил', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('штрудель', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('маасдам', case=False, na=False), 'cluster'] = 7
search_history.loc[search_history['search_query'].str.contains('эклер', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('кухонные', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('пудра', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('бабл', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('масле', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('ясно', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('гхи', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('merries', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('разделочная', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('мух', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('кухонный', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('плов', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('бутылка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('приготовления', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('mio', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('пава', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('пост', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('gillette', case=False, na=False), 'cluster'] = 20
search_history.loc[search_history['search_query'].str.contains('алиса', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('травы', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('каперсы', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('дог', case=False, na=False), 'cluster'] = 8

In [ ]:
search_history.loc[search_history['search_query'].str.contains('кето', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('терафлю', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('очиститель', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('ножи', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('шампанское', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('комфорт', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('полоски', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('бокалы', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('semper', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('черника', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('банка', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('актибио', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('пиву', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('пончики', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('помело', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('насос', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('ковер', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('кабрита', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('гренки', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('абрикос', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('ногти', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('прыщей', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('полисорб', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('корзина', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('бокс', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('аквафор', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('сервелат', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('фуэт', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('gerber', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('станция', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('теос', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('nemoloko', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('lavazza', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('вафельница', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('evervess', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('язык', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('фундук', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('бобы', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('марта', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('косметичка', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('nordic', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('спорт', case=False, na=False), 'cluster'] = 21
search_history.loc[search_history['search_query'].str.contains('робот', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('мистраль', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('окорок', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('докторская', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('дождевик', case=False, na=False), 'cluster'] = 12
search_history.loc[search_history['search_query'].str.contains('увелка', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('история', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('игры', case=False, na=False), 'cluster'] = 20
search_history.loc[search_history['search_query'].str.contains('bushido', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('burn', case=False, na=False), 'cluster'] = 5

In [ ]:
search_history.loc[search_history['search_query'].str.contains('засоров', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('сиртаки', case=False, na=False), 'cluster'] = 7
search_history.loc[search_history['search_query'].str.contains('патчи', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('крупа', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('градусник', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('hi', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('зажигалка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('пирожки', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('шницель', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('вязанка', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('имунеле', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('кус', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('чука', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('орешки', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('земля', case=False, na=False), 'cluster'] = 17
search_history.loc[search_history['search_query'].str.contains('дача', case=False, na=False), 'cluster'] = 17
search_history.loc[search_history['search_query'].str.contains('птитим', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('биг', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('крахмал', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('adrenaline', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('брауни', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('аккумулятор', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('shock', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('малютка', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('автомобильный', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('хлоргексидин', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('крылья', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('корнерсы', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('тарелки', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('заморозка', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('грейпфрут', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('солнышко', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('чевапчичи', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('баунти', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('наполеон', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('мак', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('браслет', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('лепешки', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('расческа', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('яндекс', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('rich', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('оладьи', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('диффузор', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('pepper', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('таорог', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('эконива', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('накладные', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('читос', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('тапки', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('маринованные', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('форте', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('чечил', case=False, na=False), 'cluster'] = 7
search_history.loc[search_history['search_query'].str.contains('pampers', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('нарзан', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('жареный', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('хинкали', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('окраина', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('раствор', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('кускус', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('пемолюкс', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('спички', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('acuvue', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('беби', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('малоежка', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('лечо', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('bio', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('гладильная', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('дамплинги', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('бри', case=False, na=False), 'cluster'] = 7
search_history.loc[search_history['search_query'].str.contains('доя', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('сорбет', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('чиа', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('кольцо', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('тонометр', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('nutrilak', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('блю', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('отруби', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('слойка', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('буженина', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('роутер', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('fitnesshock', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('мяч', case=False, na=False), 'cluster'] = 21
search_history.loc[search_history['search_query'].str.contains('пантенол', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('тараканов', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('запеканка', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('fit', case=False, na=False), 'cluster'] = 21
search_history.loc[search_history['search_query'].str.contains('термос', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('мирамистин', case=False, na=False), 'cluster'] = 13

In [54]:
remaining_negatives = (search_history['cluster'] == -1).sum()
print(f"Осталось строк с меткой -1: {remaining_negatives}")

Осталось строк с меткой -1: 6583297


13931972

10008829

7488206

6583297

In [15]:
search_history.loc[search_history['search_query'].str.contains('отривин', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('вырезка', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('soft', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('палыча', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('клюква', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('орео', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('открытка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('нэнни', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('nutella', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('тарелка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('apple', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('пустышка', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('помада', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('цыпленок', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('профитроли', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('миксер', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('самсунг', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('шпажки', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('joonies', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('мочалка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('чебурек', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('нектарин', case=False, na=False), 'cluster'] = 4


In [16]:
search_history.loc[search_history['search_query'].str.contains('холодец', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('шаурма', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('духи', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('нестожен', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('няня', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('фанта', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('безалкогольное', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('костюм', case=False, na=False), 'cluster'] = 12
search_history.loc[search_history['search_query'].str.contains('соломка', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('садовый', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('вкусвилл', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('тушь', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('хамон', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('сэндвич', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('гранат', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('коробка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('дорадо', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('губка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('грунт', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('цукаты', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('кресло', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('магги', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('кокос', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('утюг', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('нагетсы', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('посыпка', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('type', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('мангал', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('компот', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('planto', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('choco', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('сетка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('мохито', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('скатерть', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('форма', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('yokosun', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('чаю', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('spf', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('светодиодная', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('мацони', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('bite', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('стул', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('ламбер', case=False, na=False), 'cluster'] = 7
search_history.loc[search_history['search_query'].str.contains('redmi', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('манная', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('тряпка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('лактозы', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('копчения', case=False, na=False), 'cluster'] = 9


In [17]:
search_history.loc[search_history['search_query'].str.contains('свеча', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('минтай', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('начос', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('флешка', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('кунжут', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('камера', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('parmalat', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('батарейка', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('электрическая', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('тюльпаны', case=False, na=False), 'cluster'] = 17
search_history.loc[search_history['search_query'].str.contains('губки', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('бинт', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('huggies', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('велосипед', case=False, na=False), 'cluster'] = 21
search_history.loc[search_history['search_query'].str.contains('индей', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('сенежская', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('fleur', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('диван', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('дыня', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('крышкой', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('камамбер', case=False, na=False), 'cluster'] = 7
search_history.loc[search_history['search_query'].str.contains('alpine', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('фен', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('конструктор', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('пахлава', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('bull', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('яичный', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('самокат', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('обувь', case=False, na=False), 'cluster'] = 12
search_history.loc[search_history['search_query'].str.contains('зеркало', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('zotman', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('матча', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('пита', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('яшкино', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('дядя', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('калинов', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('шторы', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('говяжий', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('кислота', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('аккумуляторный', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('чебупели', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('ваня', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('байкал', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('hipp', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('щетка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('кисель', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('активиа', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('смузи', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('ред', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('шары', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('ессентуки', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('ролтон', case=False, na=False), 'cluster'] = 3
search_history.loc[search_history['search_query'].str.contains('электрический', case=False, na=False), 'cluster'] = 15
search_history.loc[search_history['search_query'].str.contains('чак', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('бальзам', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('желатин', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('шкаф', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('суфле', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('блокнот', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('кролик', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('коровка', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('краб', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('белизна', case=False, na=False), 'cluster'] = 10

In [23]:
search_history.loc[search_history['search_query'].str.contains('твикс', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('бритва', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('машинка', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('окрошка', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('red', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('чудо', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('снежок', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('тартале', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('аквалор', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('тест', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('одеяло', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('dr', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('безлактозное', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('вибратор', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('язык', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('разрыхлитель', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('m&m’s', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('батат', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('борщ', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('крекер', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('риет', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('косметика', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('ручка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('chikalab', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('хоеб', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('буррата', case=False, na=False), 'cluster'] = 7
search_history.loc[search_history['search_query'].str.contains('ritter', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('стекло', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('фруто', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('спрайт', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('sport', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('рикотта', case=False, na=False), 'cluster'] = 7
search_history.loc[search_history['search_query'].str.contains('курага', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('nutrilon', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('ласка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('act', case=False, na=False), 'cluster'] = 19
search_history.loc[search_history['search_query'].str.contains('synergetic', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('шпроты', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('светильник', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('растишка', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('пломбир', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('teos', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('сендвич', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('карпаччо', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('фреш', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('бон', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('canin', case=False, na=False), 'cluster'] = 8
search_history.loc[search_history['search_query'].str.contains('шампанское', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('pringles', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('тан', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('баранки', case=False, na=False), 'cluster'] = 6

In [24]:
search_history.loc[search_history['search_query'].str.contains('страчателла', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('nan', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('нан', case=False, na=False), 'cluster'] = 11
search_history.loc[search_history['search_query'].str.contains('свр', case=False, na=False), 'cluster'] = 7
search_history.loc[search_history['search_query'].str.contains('донат', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('айран', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('лепешка', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('петелинка', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('милка', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('чупа', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('маска', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('белок', case=False, na=False), 'cluster'] = 21
search_history.loc[search_history['search_query'].str.contains('барни', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('маскарпоне', case=False, na=False), 'cluster'] = 7
search_history.loc[search_history['search_query'].str.contains('моти', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('фета', case=False, na=False), 'cluster'] = 7
search_history.loc[search_history['search_query'].str.contains('актимель', case=False, na=False), 'cluster'] = 1
search_history.loc[search_history['search_query'].str.contains('добрый', case=False, na=False), 'cluster'] = 5
search_history.loc[search_history['search_query'].str.contains('коврик', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('доска', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('сигареты', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('кешью', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('скумбрия', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('желе', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('фунчоза', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('контейнер', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('глютена', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('чупс', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('snaq', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('кастрю', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('эклеры', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('лазанья', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('пончик', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('нож', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('крупа', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('фильтр', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('milka', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('мг', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('баран', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('ламп', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('тряпк', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('тофу', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('fabriq', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('маршмеллоу', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('posay', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('чокопай', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('zewa', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('бульон', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('треска', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('миндаль', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('пирож', case=False, na=False), 'cluster'] = 6
search_history.loc[search_history['search_query'].str.contains('снеки', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('ведро', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('горбуша', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('барилла', case=False, na=False), 'cluster'] = 14
search_history.loc[search_history['search_query'].str.contains('нурофен', case=False, na=False), 'cluster'] = 13
search_history.loc[search_history['search_query'].str.contains('купаты', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('ваза', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('zero', case=False, na=False), 'cluster'] = 2
search_history.loc[search_history['search_query'].str.contains('специи', case=False, na=False), 'cluster'] = 18
search_history.loc[search_history['search_query'].str.contains('карбонад', case=False, na=False), 'cluster'] = 9
search_history.loc[search_history['search_query'].str.contains('фетакса', case=False, na=False), 'cluster'] = 7
search_history.loc[search_history['search_query'].str.contains('ножницы', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('бассейн', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('пленка', case=False, na=False), 'cluster'] = 10
search_history.loc[search_history['search_query'].str.contains('сулугуни', case=False, na=False), 'cluster'] = 7
search_history.loc[search_history['search_query'].str.contains('корнишоны', case=False, na=False), 'cluster'] = 4
search_history.loc[search_history['search_query'].str.contains('royal', case=False, na=False), 'cluster'] = 8
search_history.loc[search_history['search_query'].str.contains('машина', case=False, na=False), 'cluster'] = 20


In [57]:
import os
import pandas as pd
import polars as pl 

sh_topl = pl.from_pandas(search_history)

output_dir = "cluster_search_full"
os.makedirs(output_dir, exist_ok=True)

chunk_size = 1_000_000  

num_chunks = len(sh_topl) // chunk_size + 1  

for i in range(num_chunks):
    start_idx = i * chunk_size
    end_idx = (i + 1) * chunk_size
    chunk = sh_topl[start_idx:end_idx]
    
    chunk.write_parquet(f"{output_dir}/cluster_search_part_{i}.parquet") 

In [58]:
import polars as pl
import pandas as pd
import numpy as np
import catboost
import os
from datetime import date, timedelta
from itertools import combinations

from local_utils import *
import lightgbm as lgb

%load_ext autoreload
%autoreload 2


In [71]:
test_start_date = date(2024, 8, 1)
val_start_date = date(2024, 7, 1)
val_end_date = date(2024, 7, 31)
train_end_date = date(2024, 6, 30)
data_path = "C:\\Users\\Admin\\Desktop\\AIM 2сем\\ML2\\hw2"

In [72]:
product_information = pl.read_csv(
    os.path.join(data_path, 'cluster_product_information.csv'),
    ignore_errors=True
)

In [73]:
product_information

product_id,name,brand,type,category_id,category_name,price,discount_price,cluster
i64,str,str,str,i64,str,f64,f64,i64
26176363,"""Развивающие тесты (3-4 года) (…","""Machaon""","""Печатная книга: Развитие детей""",780,"""Книги""",380.0,274.0,-1
29898500,"""Mexx Туалетная вода Ice Touch …","""Mexx""","""Туалетная вода""",117,"""Мужская""",2645.0,1859.0,10
33967827,"""64 ГБ USB Флеш-накопитель USB …","""SmartBuy""","""USB-флеш-накопитель""",178,"""Флешки и CD-R""",1690.0,469.0,15
135938830,"""Чай листовой чёрный Ahmad Tea …","""Ahmad Tea""","""Чай листовой""",465,"""Чай листовой""",319.0,244.0,5
137920686,"""Seagate 4 ТБ Внешний жесткий д…","""Seagate""","""Внешний жесткий диск""",615,"""Внешние жесткие диски""",28590.0,9539.0,15
…,…,…,…,…,…,…,…,…
1665110648,"""Салфетки бумажные 1 слоя 24 см…","""Gratias""","""Бумажные салфетки""",301,"""Салфетки бумажные""",0.0,0.0,15
1667861556,"""Зубная паста детская R.O.C.S. …","""R.O.C.S.""","""Зубная паста""",642,"""Зубные пасты и ополаскиватели""",0.0,0.0,13
1674778291,"""Доска разделочная 36x24x1,2см …","""BY""","""Разделочная доска""",202,"""Ножи и доски""",0.0,0.0,-1


In [74]:
pi_test = product_information.sample(1000)
pi_test = pi_test.to_pandas()

In [75]:
product_information = product_information.to_pandas()

In [79]:
cluster_mapping = {
    1: ['пармалат', 'теос', 'nemoloko', 'имунеле', 'таорог', 'эконива', 'гхи', 'масле', 'савушкин',
        'monterra', 'ацидофилин', 'эскимо', 'маргарин', 'viola', 'яица', 'урбеч'],
    2: ['закуска', 'жевачка', 'чтпсы', 'rex', 'любятово', 'щербет', 'mirel', 'шоколад', 'баунти', 'наполеон',
        'читос', 'fitnesshock', 'shock', 'кето', 'аленка', 'чоко', 'фри', 'бабл', 'равиоли', 'роше', 'конфитюр',
        'kitkat', 'ferrero', 'марс', 'рафаэлло', 'бенто', 'скитлс', 'milky', 'merci', 'мерси', 'twix', 'подушечки',
        'панчо', 'пикник', 'solvie', 'веган', 'крупы', 'lindt', 'роял'],
    3: ['макаруны', 'кимчи', 'клетчатка', 'псиллиум', 'nordic', 'мистраль', 'увелка', 'биг', 'птитим', 'кус',
        'корнерсы', 'чиа', 'отруби', 'кускус', 'плов', 'ясно', 'травы', 'паэлья', 'yelli', 'удон'],
    4: ['абрикосы', 'архыз', 'халапеньо', 'смородина', 'черника', 'банка', 'помело', 'грейпфрут', 'мак', 'фундук',
       'бобы', 'каперсы', 'орешки', 'чука', 'лечо', 'сорбет', 'маринованные', 'хурма', 'облепиха', 'чери', 'слива',
       'трюфель', 'ржаной', 'оливье', 'романо', 'инжир', 'крахмал'],
    5: ['borjomi', 'energy', 'мажитэль', 'nescafe', 'tea', 'greenfield', 'lavazza', 'evervess', 'adrenaline', 'шампанское',
       'бокалы', 'актибио', 'пиву', 'блю', 'pepper', 'rich', 'bushido', 'burn', 'нарзан', 'швепс', 'дюшес', 'пепси',
       'lipton', 'новотерская', 'лёд', 'drip'],
    6: ['савоярди', 'эклер', 'пончики', 'гренки', 'слойка', 'брауни', 'лепешки', 'пирожки', 'безе', 'запеканка',
       'оладьи', 'пергамент', 'ватрушка', 'круасан', 'галеты', 'коржи', 'хачапури', 'край'],
    7: ['natura', 'маасдам', 'сиртаки', 'чечил', 'бри', 'альметте', 'халуми'],
    8: ['pi', 'дог', 'purina', 'one', 'лоток', 'миска', 'cat', 'роял'],
    9: ['тефтели', 'перекус', 'сибас', 'телятина', 'штрудель', 'окорок', 'докторская', 'язык', 'жареный', 'хинкали',
       'окраина', 'вязанка', 'шницель', 'дамплинги', 'крылья', 'сервелат', 'фуэт', 'заморозка', 'чевапчичи', 'hi',
       'буженина', 'гуляш', 'палтус', 'велком', 'панировка', 'балык', 'перепелка', 'черкизово', 'maggi', 'обед',
       'бульмени', 'манты', 'килька', 'крылышки', 'прошутто', 'сальчичон', 'состски'],
    10: ['папка', 'liebe', 'фломастеры', 'ваниш', 'мешок', 'синергетик', 'таз', 'жалюзи', 'сады', 'обои', 'фигурки',
        'маркер', 'лак', 'тазик', 'листы', 'шланг', 'брелок', 'кассеты', 'органайзер', 'полка', 'вернель', 'мебель',
        'кухонные', 'разделочная', 'мух', 'кухонный', 'бутылка', 'приготовления', 'пост', 'гладильная', 'тарелки',
        'корзина', 'бокс', 'расческа', 'термос', 'пемолюкс', 'спички', 'зажигалка', 'насос', 'ковер', 'очиститель',
        'ножи', 'блюда', 'труб', 'надувной', 'grass', 'пила', 'комод', 'садовая', 'история', 'засоров', 'тапки',
        'кольцо', 'тараканов', 'инструменты', 'фейри', 'вешалки', 'горшок', 'розетка', 'вешалка', 'крючки', 'азелит',
        'санокс', 'конверт', 'fairy', 'ручки', 'терка', 'сердечки', 'фумигатор', 'ушастый', 'веник', 'диск', 'чашка',
        'массажер', 'лопатка', 'ежедневник', 'насадка', 'коробки', 'розжиг', 'газовая', 'силит', 'персил', 'kix',
        'мешки', 'domestos', 'казан', 'туалет', 'простынь', 'bang', 'монитор'],
    11: ['lol', 'мама', 'kabrita', 'кубики', 'momi', 'baby', 'semper', 'кабрита', 'малютка', 'nutrilak', 'pampers',
        'беби', 'малоежка', 'gerber', 'коляска', 'merries', 'хот', 'комфорт', 'поп', 'similac', 'пазл', 'нянь',
        'lovular', 'горки', 'машинки', 'шар', 'домик', 'пазлы'],
    12: ['дождевик', 'nike'],
    13: ['ти', 'кашля', 'just', 'эвалар', 'ополаскиватель', 'термометр', 'снуп', 'полоски', 'хлоргексидин', 'солнышко',
        'пантенол', 'мирамистин', 'раствор', 'acuvue', 'bio', 'форте', 'терафлю', 'полисорб', 'прыщей', 'аквафор',
        'тонометр', 'градусник', 'смекта', 'эспумизан', 'ромашка', 'мелатонин', 'лекарства', 'доктор', 'стрепсилс',
        'лисички', 'таблетница', 'аптека'],
    14: ['heinz', 'тар', 'махеев', 'шрирача'],
    15: ['аккумуляторная', 'отпариватель', 'mio', 'алиса', 'станция', 'вафельница', 'роутер', 'яндекс', 'диффузор',
        'робот', 'автомобильный', 'фона', 'infinix', 'тостер', 'hdmi', 'аккумулятор', 'ирригатор', 'шуруповерт',
        'триммер', 'капучинатор', 'электронная', 'аппарат', 'монитор'],
    17: ['земля', 'дача'],
    19: ['green', 'dove', 'лепестки', 'чистая', 'лубрикант', 'пудра', 'серьги', 'карандаш', 'накладные', 'косметичка',
        'браслет', 'доя', 'ногти', 'линия', 'bioderma', 'nivea', 'dolce', 'head', 'ice', 'matrix', 'консилер',
        'анальная', 'спирт', 'holika', 'член', '18+', 'estel', 'rocher', 'пробка', 'mixit', 'vichy', 'pusy',
        'aravia'],
    20: ['табачные', 'авто', 'gillette', 'игры', 'автомобиль', 'дорожный', 'vaz', 'сигарета', 'фигурка'],
    21: ['korner', 'стевия', 'креатин', 'спорт', 'fit', 'мяч', 'fitnesshock', 'гантели', 'шейкер']
}

In [80]:
import re

for cluster_id, keywords in cluster_mapping.items():
    pattern = '|'.join(map(re.escape, keywords))
    product_information.loc[product_information['name'].str.contains(pattern, case=False, na=False), 'cluster'] = cluster_id

In [81]:
product_information[product_information['cluster'] == -1]

,product_id,name,brand,type,category_id,category_name,price,discount_price,cluster
0,26176363,Развивающие тесты (3-4 года) (нов.обл.) | Земц...,Machaon,Печатная книга: Развитие детей,780,Книги,380.0,274.0,-1
39,200860750,Справочник оттенков RAL Classic K7,RAL,Каталог отделочных материалов,320,Строительство и ремонт,1550.0,1550.0,-1
185,1474404727,"Мини-печь духовка Lumme LU-1707 38 литров, 3 р...",Lumme,Мини-печь,537,Микроволновые печи,32450.0,6566.0,-1
228,1574023553,Гирлянда светодиодная Штора Занавес 3х3 м золо...,TEWSON,Электрогирлянда,235,Украшения,3500.0,108.0,-1
249,1604899193,"ELCASA Набор корзин длина 35 см, ширина 24.5 с...",ELCASA,Набор корзин,553,Коробки и товары для хранения,7490.0,1438.0,-1
...,...,...,...,...,...,...,...,...,...
238256,149095472,Набор игровой Плей-До Кран-погрузчик PLAY-DOH ...,Hasbro,Масса для лепки,385,Лепка,1956.0,1279.0,-1
238286,242250865,Покрывало Arya Жаккард с кружевом 240X260 Paisley,ARYA,Плед,345,Текстиль,27990.0,2237.0,-1
238297,306025378,Реверсивный рубильник модульный (трехпозиционн...,МЕАНДР,Переключатель,320,Строительство и ремонт,1664.0,1664.0,-1
238315,484616725,"Физалис, 100 г",Нет бренда,Фрукты,799,Экзотические фрукты,299.0,299.0,-1


In [82]:
product_information.to_csv('cluster_product_information_full.csv', index=False)

In [85]:
ss = pl.read_csv(
    os.path.join(data_path, 'cluster_product_information_full.csv'),
    ignore_errors=True
)
ss = ss.to_pandas()
ss[ss['cluster'] == -1]

,product_id,name,brand,type,category_id,category_name,price,discount_price,cluster
0,26176363,Развивающие тесты (3-4 года) (нов.обл.) | Земц...,Machaon,Печатная книга: Развитие детей,780,Книги,380.0,274.0,-1
39,200860750,Справочник оттенков RAL Classic K7,RAL,Каталог отделочных материалов,320,Строительство и ремонт,1550.0,1550.0,-1
185,1474404727,"Мини-печь духовка Lumme LU-1707 38 литров, 3 р...",Lumme,Мини-печь,537,Микроволновые печи,32450.0,6566.0,-1
228,1574023553,Гирлянда светодиодная Штора Занавес 3х3 м золо...,TEWSON,Электрогирлянда,235,Украшения,3500.0,108.0,-1
249,1604899193,"ELCASA Набор корзин длина 35 см, ширина 24.5 с...",ELCASA,Набор корзин,553,Коробки и товары для хранения,7490.0,1438.0,-1
...,...,...,...,...,...,...,...,...,...
238256,149095472,Набор игровой Плей-До Кран-погрузчик PLAY-DOH ...,Hasbro,Масса для лепки,385,Лепка,1956.0,1279.0,-1
238286,242250865,Покрывало Arya Жаккард с кружевом 240X260 Paisley,ARYA,Плед,345,Текстиль,27990.0,2237.0,-1
238297,306025378,Реверсивный рубильник модульный (трехпозиционн...,МЕАНДР,Переключатель,320,Строительство и ремонт,1664.0,1664.0,-1
238315,484616725,"Физалис, 100 г",Нет бренда,Фрукты,799,Экзотические фрукты,299.0,299.0,-1
